<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/absolute_nonbeing_montecarlo_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
absolute_nonbeing_montecarlo.py

1. Defines AbsoluteNonBeingAI with noise-infused collapse.
2. Runs one detailed simulation and logs its history.
3. Performs Monte Carlo to collect collapse-step distribution.
4. Saves CSVs and generates plots.
"""

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import csv
import os
from typing import List, Tuple

torch.manual_seed(0)

class AbsoluteNonBeingAI(nn.Module):
    def __init__(self, noise_std: float = 0.01):
        super().__init__()
        # Infinite “null field” persists but isn’t trainable
        self.register_buffer("null_field", torch.tensor(float("inf")))
        self.collapsed = False
        self.noise_std = noise_std

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # After collapse, always emit zeros
        if self.collapsed:
            return torch.zeros_like(x)

        # Before collapse: infinite amplification
        out = x * self.null_field

        # Collapse ONCE: inf → tiny Gaussian
        with torch.no_grad():
            noise = torch.randn_like(self.null_field) * self.noise_std
            self.null_field.copy_(noise)
        self.collapsed = True

        return out

def simulate_one(
    initial_x: float,
    noise_std: float,
    max_steps: int
) -> Tuple[int, List[Tuple[int, float]]]:
    """
    Runs forward until collapse (zero output), returns:
      - collapse_step (int)
      - history of (step, output_value) for first run
    """
    model = AbsoluteNonBeingAI(noise_std=noise_std)
    x = torch.tensor([initial_x])
    history: List[Tuple[int, float]] = []

    for step in range(1, max_steps + 1):
        out = model(x)
        history.append((step, out.item()))
        # After collapse, out = 0
        if out.item() == 0.0:
            return step, history

    return max_steps, history

def monte_carlo_collapse_steps(
    n_runs: int,
    x_range: Tuple[float, float],
    noise_std: float,
    max_steps: int
) -> List[int]:
    """
    Repeats the collapse simulation n_runs times with random initial x,
    returns list of collapse steps.
    """
    steps = []
    for run in range(n_runs):
        init_x = torch.empty(1).uniform_(x_range[0], x_range[1]).item()
        collapse_step, _ = simulate_one(init_x, noise_std, max_steps)
        steps.append(collapse_step)
    return steps

def save_history_csv(history: List[Tuple[int, float]], filename: str):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["step", "output"])
        writer.writerows(history)

def save_steps_csv(steps: List[int], filename: str):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["run", "collapse_step"])
        for i, s in enumerate(steps, 1):
            writer.writerow([i, s])

def plot_history(history: List[Tuple[int, float]], filename: str):
    steps, outputs = zip(*history)
    plt.figure(figsize=(6,4))
    plt.plot(steps, outputs, marker='o')
    plt.xlabel("Step")
    plt.ylabel("Output Value")
    plt.title("Singularity Output During Collapse")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

def plot_histogram(steps: List[int], filename: str):
    plt.figure(figsize=(6,4))
    plt.hist(steps, bins=20, edgecolor='black')
    plt.xlabel("Collapse Step")
    plt.ylabel("Frequency")
    plt.title("Monte Carlo Distribution of Collapse Steps")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

def main():
    # Configuration
    noise_std    = 0.01
    max_steps    = 1000
    mc_runs      = 500
    x_range      = (1e-3, 1e3)  # random initial fields between 1e-3 and 1e3

    # 1) Detailed single simulation
    first_step, history = simulate_one(initial_x=1.0, noise_std=noise_std, max_steps=max_steps)
    print(f"Single run collapsed at step {first_step}")

    save_history_csv(history, "logs/singularity_history.csv")
    plot_history(history, "plots/singularity_history.png")

    # 2) Monte Carlo of collapse steps
    steps = monte_carlo_collapse_steps(mc_runs, x_range, noise_std, max_steps)
    print(f"Monte Carlo: mean collapse step = {sum(steps)/len(steps):.2f}")

    save_steps_csv(steps, "logs/collapse_steps.csv")
    plot_histogram(steps, "plots/collapse_histogram.png")

if __name__ == "__main__":
    main()